In [7]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

In [8]:
import os
from os import getenv
from langfuse import Langfuse
from dotenv import find_dotenv, load_dotenv
from src.workflow import AssistantFlow
from deepeval import evaluate
from deepeval.test_case import LLMTestCase
from deepeval.metrics import AnswerRelevancyMetric

In [ ]:
if not os.getenv("PROD"):
    load_dotenv(find_dotenv())


os.environ["OPENAI_API_KEY"] = ""

In [10]:
langfuse = Langfuse(
    public_key=getenv("LANGFUSE_PUBLIC_KEY"),
    secret_key=getenv("LANGFUSE_SECRET_KEY"),
    host=getenv("LANGFUSE_HOST"),
)

In [11]:
import time


dataset = langfuse.get_dataset("qa_for_test")

result = None
for item in dataset.items:
    try:
        with item.observe(
            run_name="Metric compute IlyaGusev/saiga_nemo_12b",
            run_metadata={"model": "gpt-4o"},
        ) as trace_id:
            w = AssistantFlow(timeout=3 * 60)

            output = await w.run(query=item.input)

            test_case = LLMTestCase(
                input=item.input,
                actual_output=item.expected_output,
            )

            answer_relevancy_metric = AnswerRelevancyMetric(
                model="gpt-4o", include_reason=False
            )

            result = evaluate(test_cases=[test_case], metrics=[answer_relevancy_metric], print_results=False)
            
            # print(result[0].metrics_data[0].score)
            # break

            langfuse.score(
                trace_id=trace_id,
                name="AnswerRelevancy",
                value=result.test_results[0].metrics_data[0].score,
            )
            
            # time.sleep(1)
    except Exception as e:
        print(e)


langfuse.flush()

MESSAGES
[{'role': 'user', 'content': 'Твоя задача - определить, релевантны ли предоставленные документы запросу пользователя. Релевантым считай тот документ, в котором тема хотя бы смежно связана с запросом. Верни ровно один массив из строк \'0\' или \'1\', где \'1\' означает, что документ релевантен запросу, а \'0\' - что нерелевантен. Массив должен иметь ровно столько элементов, сколько документов в запросе.\n\nЗапрос: \'сделать копию система электронного документооборота\'\n\nДокументы:\nДокумент 0:\nВопрос: скачать копию доверенности через sed\nОтвет: копию доверенности можно выгрузить через раздел отчеты в sed. также там можно посмотреть номер и срок действия доверенности.\n\nДокумент 1:\nВопрос: получение электронная цифровая подпись\nОтвет: прежде чем создавать заявку, пожалуйста попробуйте самостоятельно выпустить эцп на странице "электронные подписи" в личном разделе лк\n\nДокумент 2:\nВопрос: как скачать скан доверенности?\nОтвет: скан-копию доверенности можно выгрузить в ли

✨ You're running DeepEval's latest Answer Relevancy Metric! (using gpt-4o, strict=False, async_mode=True)...

Event loop is already running. Applying nest_asyncio patch to allow async execution...


Evaluating 1 test case(s) in parallel: |          |  0% (0/1) [Time Taken: 00:00, ?test case/s]WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
E0000 00:00:1742020618.371882  436442 ssl_transport_security_utils.cc:114] Corruption detected.
E0000 00:00:1742020618.371976  436442 ssl_transport_security_utils.cc:71] error:1e000065:Cipher functions:OPENSSL_internal:BAD_DECRYPT
E0000 00:00:1742020618.371987  436442 ssl_transport_security_utils.cc:71] error:1000008b:SSL routines:OPENSSL_internal:DECRYPTION_FAILED_OR_BAD_RECORD_MAC
E0000 00:00:1742020618.377453  436442 secure_endpoint.cc:298] Decryption error: TSI_DATA_CORRUPTED
E0000 00:00:1742020618.378042  436442 ssl_transport_security_utils.cc:88] SSL_write failed with error SSL_ERROR_SSL
Evaluating 1 test case(s) in parallel: |██████████|100% (1/1) [Time Taken: 00:06,  6.25s/test case]


✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.

...

MESSAGES
[{'role': 'user', 'content': "Твоя задача - определить, релевантны ли предоставленные документы запросу пользователя. Релевантым считай тот документ, в котором тема хотя бы смежно связана с запросом. Верни ровно один массив из строк '0' или '1', где '1' означает, что документ релевантен запросу, а '0' - что нерелевантен. Массив должен иметь ровно столько элементов, сколько документов в запросе.\n\nЗапрос: 'не получается зайти в личный кабинет'\n\nДокументы:\nДокумент 0:\nВопрос: не получается зайти в личный кабинет\nОтвет: при проблемах со входом в личный кабинет, прежде чем создавать заявку в поддержку, убедитесь, что заходите в лк на сайте link указываете актуальные и верные логин и пароль. если вам неизвестен логин, обратитесь к руководителю (дм), он сможет посмотреть ваш логин и сбросить пароль в веб-табеле. для самостоятельного сброса пароля позвоните с вашего мобильного телефона на phone, наберите добавочный номер 10100, нажмите * и подтвердите сброс пароля, нажав #. обн

✨ You're running DeepEval's latest Answer Relevancy Metric! (using gpt-4o, strict=False, async_mode=True)...

Event loop is already running. Applying nest_asyncio patch to allow async execution...


Evaluating 1 test case(s) in parallel: |██████████|100% (1/1) [Time Taken: 00:09,  9.99s/test case]


✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.

In [12]:
# from datetime import datetime


# traces = langfuse.fetch_observations(
#     from_start_time=datetime(2025, 3, 14, 19, 0, 42),
#     to_start_time=datetime(2025, 3, 14, 19, 24, 12),
# )

# traces